# Example Usage

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
from IPython.display import display, Markdown
def printmd(object):
    display(Markdown(str(object).replace('\n', '<br>')))

In [ ]:
import chalc as ch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
import h5py, logging, sys

# Disable some deprecation warnings from the seaborn plotting library
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.captureWarnings(True)

For our data we sample 100 points on a circle with some noise and 100 points from inside the unit disk.

In [ ]:
rng = np.random.default_rng(40)
num_points_circle = 100
num_points_disk = 100
mean = [0, 0]
cov = np.eye(2)*0.01
circle = np.array([[np.sin(2*np.pi*t), np.cos(2*np.pi*t)] for t in rng.random(num_points_circle)]).T +\
    rng.multivariate_normal(mean, cov, num_points_circle).T # points as columns
disk = np.sqrt(rng.random(num_points_disk)) * np.array([[np.sin(2*np.pi*t), np.cos(2*np.pi*t)] for t in rng.random(num_points_disk)]).T
points = np.concatenate((circle, disk), axis=1)
plt.scatter(circle[0, :], circle[1, :], s=10)
plt.scatter(disk[0, :], disk[1, :], s=10)
plt.gca().set_aspect('equal')
colours = [0]*num_points_circle + [1]*num_points_disk
plt.title('Point cloud')
plt.show()

In [ ]:
K, _ = ch.chromatic.alpha(points, colours)
printmd(f'$K$ has {len(K.simplices[1])} 1-simplices and {len(K.simplices[2])} 2-simplices')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 4))
ch.plotting.draw_filtration(K, points, time=0.3, include_colours=[0], ax=ax[0])
ax[0].set_title('Colour 0')
ch.plotting.draw_filtration(K, points, time=0.3, include_colours=[1], ax=ax[1])
ax[1].set_title('Colour 1')
ch.plotting.draw_filtration(K, points, time=0.3, ax=ax[2])
ax[2].set_title('Both colours')
for i in range(3):
    ax[i].set_xlim(-1.2, 1.2)
    ax[i].set_ylim(-1.2, 1.2)
fig.suptitle('Complexes at $t=0.3$')
plt.tight_layout()
plt.show()

In [ ]:
# using the previously computed filtration
dgms_alpha = ch.sixpack.from_filtration(K, dom=[0])
# or directly from the point cloud
dgms_delcech = ch.sixpack.compute(points, colours, dom=[0], method="delcech")
dgms_delrips = ch.sixpack.compute(points, colours, dom=[0], method="delrips")

In [ ]:
print(f"The diagram names are: {list(dgms_alpha.keys())}")

# iterate over the diagrams like in a dictionary
for diagram_name, diagram in dgms_alpha:
    print(diagram_name)
    # do something with the diagram

# access individual diagrams by indexing
ker = dgms_alpha['ker']

In [ ]:
print(dgms_alpha['ker'])

In [ ]:
# check if the diagram is empty
print("The kernel is non-empty." if dgms_alpha['ker'] else "The kernel is empty.")
# or you can use bool(dgms_alpha.ker)

# __len__()
# get the number of features in the diagram 
print(f"The cokernel has {len(dgms_alpha['cok'])} features.")

# __contains__()
# check if a feature exists in a diagram
print("The domain contains a feature represented by the simplex pair (20, 40): "
      f"{(20, 40) in dgms_alpha['dom']}")

# __iter__()
# iterate over the paired and unpaired simplices together
for feature in dgms_alpha['ker']:
    if isinstance(feature, tuple):
        sigma, tau = feature
        # do something with a pair of simplices
        ...
    else:
        sigma = feature
        # do something with an unpaired simplex
        ...

# the paired and unpaired simplices can be considered separately
print("The \"paired\" property of the kernel has type: "
      f"{type(dgms_alpha['ker'].paired)}")

print("The \"unpaired\" property of the kernel has type: "
      f"{type(dgms_alpha['ker'].unpaired)}")

for sigma, tau in dgms_alpha['ker'].paired:
    # do something with paired simplices
    ...

for sigma in dgms_alpha['dom'].unpaired:
    # do something with unpaired simplices
    ...

In [ ]:
np.set_printoptions(threshold=10)
print(dgms_alpha.get_matrix('dom', 0)) # get the domain in dimension zero
# dgms_alpha.get('ker', [0, 1]) to get the kernel in dimensions zero and one
# dgms_alpha.get('ker') to get the kernel in all dimensions from zero to max(dgms_alpha.dimensions)

In [ ]:
print(dgms_alpha.entrance_times)
print(dgms_alpha.dimensions)

In [ ]:
fig1, ax1 = ch.plotting.plot_sixpack(dgms_alpha, tolerance = 1e-3)
fig1.suptitle('Chromatic Alpha')
fig1.set_figwidth(15)
fig1.set_figheight(9)
fig1.subplots_adjust(top=0.92)
plt.show()
fig2, ax2 = plt.subplots(1, 3)
# You can specify the dimensions of features to include in the plot.
ch.plotting.plot_diagram(dgms_alpha, 'rel', dimensions = {0, 2}, truncation = 0.3, ax = ax2[0], tolerance = 1e-3)
ax2[0].set_title('Chromatic Alpha')
# You can also specify a single dimension as an integer.
ch.plotting.plot_diagram(dgms_delcech, 'rel', dimensions = 1, truncation = 0.3, ax = ax2[1], tolerance = 1e-3)
ax2[1].set_title('Chromatic Delaunay-Cech')
# If no dimensions are specified, all features will be included.
ch.plotting.plot_diagram(dgms_delrips, 'rel', truncation = 0.3, ax = ax2[2], tolerance = 1e-3)
ax2[2].set_title('Chromatic Delaunay-Rips')
fig2.suptitle('Individual relative diagrams')
fig2.set_figwidth(15)
plt.show()

We can save the diagrams to a HDF5 file or load a diagram from a HDF5 file.
The HDF5 format is 

In [ ]:
with h5py.File('test.h5', 'w') as f:
    dgms_alpha.save(f)

with h5py.File('test.h5', 'r') as f:
    dgms_alpha_from_file = ch.sixpack.DiagramEnsemble.from_file(f)

print(dgms_alpha_from_file == dgms_alpha)

We can visualise the 2-skeleton of the filtration for points in 2D:

In [ ]:
animation = ch.plotting.animate_filtration(
    K, points, filtration_times=list(np.linspace(0, 1.0, 45)), animation_length=5)
animation